# Partie 1 : Création du dataset

In [1]:
import os
from keras.preprocessing import image

folders = os.listdir('/dataset')

image_data = []
labels = []
count = 0

for ix in folders:
    path = os.path.join("/dataset", ix)
    print(path, count) 
    if(path!="/dataset/.ipynb_checkpoints"):
      for im in os.listdir(path):
          try:
              img = image.load_img(os.path.join(path,im), target_size = (224,224))
              img_array = image.img_to_array(img)
              image_data.append(img_array)
              labels.append(count)
          except:
              pass
      count += 1


/content/drive/MyDrive/Dataset/Pikachu 0


/usr/local/lib/python3.7/dist-packages/PIL/Image.py:960: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  "Palette images with Transparency expressed in bytes should be "


/content/drive/MyDrive/Dataset/Salamèche 1
/content/drive/MyDrive/Dataset/Ptéra 2
/content/drive/MyDrive/Dataset/Psykokwak 3
/content/drive/MyDrive/Dataset/Miaouss 4
/content/drive/MyDrive/Dataset/Rapasdepic 5
/content/drive/MyDrive/Dataset/Minidraco 6
/content/drive/MyDrive/Dataset/Piafabec 7
/content/drive/MyDrive/Dataset/Carapuce 8
/content/drive/MyDrive/Dataset/Bulbizarre 9


In [2]:
#Ici on melange le dataset

import random

combined_dataset = list(zip(image_data, labels))
random.shuffle(combined_dataset)
image_data[:], labels[:] = zip(*combined_dataset)

In [3]:
# Convertissement du dataset en array

import numpy as np
from keras.utils import np_utils
X_train = np.array(image_data)
Y_train = np.array(labels)
Y_train = np_utils.to_categorical(Y_train)

# Partie 2 : modele ResNet50

In [4]:
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import *
from tensorflow.keras.models import Model
import numpy as np
print("Imported Successfully!")

Imported Successfully!


In [5]:
model = ResNet50(include_top = False, weights = 'imagenet', input_shape = (224,224,3)) #False car on donne au model notre propre classificateur
print(model.summary())

94781440/94765736 [==============================] - 0s 0us/step
Model: "resnet50"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv1_pad (ZeroPadding2D)      (None, 230, 230, 3)  0           ['input_1[0][0]']                
                                                                                                  
 conv1_conv (Conv2D)            (None, 112, 112, 64  9472        ['conv1_pad[0][0]']              
                                )                                                                 
                          

Création de notre propre classificateur

In [6]:
av1 = GlobalAveragePooling2D()(model.output)
fc1 = Dense(256, activation = 'relu')(av1) #Dense layer de 256 neurones et relu comme fonction d'activation
d1 = Dropout(0.5)(fc1) #Dropout de 0.5 veut dire que certain neurones serons aleatoirment non activés afin d'eviter le surapprentissage 
fc2 = Dense(10, activation = 'softmax')(d1) #nb de classe d'images / nb de Pokemon

Connection entre ResNet50 et le classificateur

In [7]:
model_new = Model(inputs = model.input, outputs = fc2) # Model() est une API de Keras, afin de definir les input et output  du model
model_new.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv1_pad (ZeroPadding2D)      (None, 230, 230, 3)  0           ['input_1[0][0]']                
                                                                                                  
 conv1_conv (Conv2D)            (None, 112, 112, 64  9472        ['conv1_pad[0][0]']              
                                )                                                                 
                                                                                              

# Partie 3 : Prediction

In [8]:
#premiere prediction

from tensorflow.keras.applications.resnet50 import preprocess_input

image_path = 'salameche2.jpg'
img = image.load_img(image_path,target_size = (224,224))
x = image.img_to_array(img)
x = np.expand_dims(x,axis=0)
x = preprocess_input(x)

pred = model_new.predict(x)
print(np.argmax(pred))

0


In [9]:
#compilation du nouveau modele

adam = Adam(lr = 0.00003)
model_new.compile(loss = 'categorical_crossentropy', optimizer = adam, metrics = ['accuracy'])

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


In [10]:
for ix in range(len(model_new.layers)):
    print(ix, model_new.layers[ix])

0 <keras.engine.input_layer.InputLayer object at 0x7f048ab2f790>
1 <keras.layers.convolutional.ZeroPadding2D object at 0x7f0489116d50>
2 <keras.layers.convolutional.Conv2D object at 0x7f04909e2690>
3 <keras.layers.normalization.batch_normalization.BatchNormalization object at 0x7f0488d21c10>
4 <keras.layers.core.activation.Activation object at 0x7f0488d21bd0>
5 <keras.layers.convolutional.ZeroPadding2D object at 0x7f048845a1d0>
6 <keras.layers.pooling.MaxPooling2D object at 0x7f0484b64a50>
7 <keras.layers.convolutional.Conv2D object at 0x7f0484b736d0>
8 <keras.layers.normalization.batch_normalization.BatchNormalization object at 0x7f0484b6c090>
9 <keras.layers.core.activation.Activation object at 0x7f0484b88210>
10 <keras.layers.convolutional.Conv2D object at 0x7f0488d21ed0>
11 <keras.layers.normalization.batch_normalization.BatchNormalization object at 0x7f0484b8bf50>
12 <keras.layers.core.activation.Activation object at 0x7f0484b1a210>
13 <keras.layers.convolutional.Conv2D object at 

In [11]:
#affinage du modele

for ix in range(169):
    model_new.layers[ix].trainable = False
print(model_new.summary())

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv1_pad (ZeroPadding2D)      (None, 230, 230, 3)  0           ['input_1[0][0]']                
                                                                                                  
 conv1_conv (Conv2D)            (None, 112, 112, 64  9472        ['conv1_pad[0][0]']              
                                )                                                                 
                                                                                              

In [12]:
#entrainement du modele

hist = model_new.fit(X_train, Y_train, shuffle = True, batch_size = 16, epochs = 8, validation_split = 0.20)

Epoch 1/8
83/83 [==============================] - 964s 12s/step - loss: 1.7482 - accuracy: 0.4369 - val_loss: 0.8469 - val_accuracy: 0.7143
Epoch 2/8
83/83 [==============================] - 942s 11s/step - loss: 0.6288 - accuracy: 0.8229 - val_loss: 0.3814 - val_accuracy: 0.8754
Epoch 3/8
83/83 [==============================] - 943s 11s/step - loss: 0.2949 - accuracy: 0.9331 - val_loss: 0.2508 - val_accuracy: 0.9240
Epoch 4/8
83/83 [==============================] - 941s 11s/step - loss: 0.1790 - accuracy: 0.9620 - val_loss: 0.2021 - val_accuracy: 0.9301
Epoch 5/8
83/83 [==============================] - 951s 11s/step - loss: 0.1035 - accuracy: 0.9802 - val_loss: 0.1886 - val_accuracy: 0.9331
Epoch 6/8
83/83 [==============================] - 946s 11s/step - loss: 0.0832 - accuracy: 0.9833 - val_loss: 0.1955 - val_accuracy: 0.9362
Epoch 7/8
83/83 [==============================] - 957s 12s/step - loss: 0.0670 - accuracy: 0.9840 - val_loss: 0.1928 - val_accuracy: 0.9301
Epoch 8/8
83/

In [19]:
image_path = 'carapuce.png'
img = image.load_img(image_path,target_size = (224,224))
x = image.img_to_array(img)
x = np.expand_dims(x,axis=0)
x = preprocess_input(x)

pred = model_new.predict(x)
print(np.argmax(pred))

8
